# Objetivo

El objetivo de este cuaderno es complementar la sesión de bayesiana y asentar los conceptos más aplicables:

1. Comparación de muestras
1. Inferencia de los parámetros de una distribución



# Comparación de muestras

La comparación de muestras extrae las distribuciones que están debajo de dos o más juegos de datos y compara valores extraídos de estas distribuciones para determinar cuál es "mejor".

Normalmente empezaríamos la comparación de muestras con el paso de inferencia de la distribución, en este caso supondremos que ya conocemso la distribución que siguen los datos y solo compararemos sus resultados.


### Función Auxiliar

**¡¡Esta es una función auxiliar, no forma parte del ejercicio, deberíais poder leerla pero no se os pide que la escribáis!!**


In [1]:
# Esto está para ayudaros, no para que lo escribáis

import seaborn as sns
import matplotlib
from matplotlib import pyplot as plt

matplotlib.rcParams['figure.figsize'] = [16,8]



colores = ['orange', 'lightblue', 'lightgreen', 'green', 'red']

def dibujar_muestras(traza, nombres, res=None, datos=None):
    plt.figure()
    for i,x in enumerate([x for x in nombres if x != 'delta']):
        print(f'dibujando histograma para {x}')
        sns.histplot(traza.posterior[x].to_series(), color=colores[i])
    
    if 'delta' in nombres:
        plt.figure()
        delta = traza.posterior['delta'].to_series()
        delta_pos = delta[delta>0]
        delta_neg = delta[delta<0]
        sns.histplot(delta_neg, color='red')
        sns.histplot(delta_pos, color='lightgreen')
    
    if res:
        plt.figure()
        sns.histplot(np.concatenate(res['obs_2'])[:len(datos)], color='lightgreen')
        sns.histplot(datos, color='orange')


## Ejemplo 1 comparación de muestras uniformes
Este ejercicio es el caso más sencillo de comparación de muestras, vamos a partir de un ejemplo resuelto en el que generamos los datos dentro de PyMC3 para comparar dos muestras uniformes.

Tenemos una muestra que varía entre 20 y 30 y otra que varía entre 30 y 40, vamos a obtener valores de ambas y comparar los datos.

Para ayudarnos tenemos una función "dibujar muestras que nos ayuda a ver la forma de las distribuciones en formato Seaborn.

In [ ]:
import pymc3 as pm
import arviz as az
import numpy as np

with pm.Model() as model:
    dist_1 = pm.Uniform('dist_1', lower=20, upper=30)
    dist_2 = pm.Uniform('dist_2', lower=30, upper=40)
    
    delta = pm.Deterministic('delta', dist_2 - dist_1)
    
    trace = pm.sample(draws = 10000, return_inferencedata=True)

    az.plot_posterior(trace, figsize=(16,8))
    
dibujar_muestras(trace, ['dist_1', 'dist_2', 'delta'])

## Ejercicio 1 - Comparación de muestras uniformes

Partiendo del ejemplo de código anterior, obtén muestras para una distribución uniforme que vaya desde 10 hasta 15 y otra que vaya de 13 a 16.

¿Podrías decir que una es mejor que otra fácilmente?

In [ ]:
import pymc3 as pm
import arviz as az
import numpy as np

with pm.Model() as model:
    dist_1 = pm.Uniform('dist_1', lower=10, upper=15)
    dist_2 = pm.Uniform('dist_2', lower=13, upper=16)
    
    delta = pm.Deterministic('delta', dist_2-dist_1)
    
    trace = pm.sample(return_inferencedata=True)

    az.plot_posterior(trace, figsize=(16,8))
    
dibujar_muestras(trace, ['dist_1', 'dist_2', 'delta'])

In [ ]:
[trace.posterior['delta'].to_series().min(), trace.posterior['delta'].to_series().max()]

In [ ]:
dibujar_muestras(trace, ['dist_1', 'dist_2', 'delta'])

## Ejemplo 2 - Comparación de muestras de diferentes distribuciones

Podemos utilizar el mismo método para comparar muestras que provienen de diferentes distribuciones.

Ojo!, si tenemos un mismo proceso en el que hacemos cambios, la distribución normalmente será la misma, cuando comparamos distribuciones diferenes estamos haciendo algo muy diferente.

Un ejemplo de comparación de muestras que provienen de diferentes distribuciones puede ser comparar las visitas a página web que conseguimos desde una campaña de publicidad a través de influencers en instagram con la que hacemos a través de Google Adwords, en estos casos no está garantizado que ambas tengan la misma distribución de probabilidad subyacente.


Fijaos que la distribución normal acepta **DOS PARÁMETROS** pero no siguen el mismo criterio que la uniforme, en la uniforme le establecemos un límite inferior y superior, en la normal damos media y desviación

In [ ]:
import pymc3 as pm
import arviz as az
import numpy as np

with pm.Model() as model:
    dist_1 = pm.Uniform('dist_1', lower=5, upper=15)
    dist_2 = pm.Normal('dist_2', mu=10, sigma=3)
    
    delta = pm.Deterministic('delta', dist_2 - dist_1)
    
    trace = pm.sample(return_inferencedata=True)

    az.plot_posterior(trace, figsize=(16,8))
    
dibujar_muestras(trace, ['dist_1', 'dist_2', 'delta'])

## Ejercicio 2 - Compara una muestra normal con una muestra uniforme

Repite el ejercicio anterior para generar una muestra normal y una uniforme, queremos que la muestra normal sea claramente superior a la uniforme, prueba algunos parámetros para conseguir ese objetivo


In [ ]:
import pymc3 as pm
import arviz as az
import numpy as np

with pm.Model() as model:
    dist_1 = pm.Uniform('dist_1', 5,15 )
    dist_2 = pm.Normal('dist_2', 15, 2)
    
    delta = pm.Deterministic('delta', dist_2 - dist_1)
    
    trace = pm.sample(return_inferencedata=True)

    az.plot_posterior(trace, figsize=(16,8))
    
dibujar_muestras(trace, ['dist_1', 'dist_2', 'delta'])

## Ejercicio 3 - Distribución de Poisson vs uniforme

Cambia el ejercicio anterior para que, en lugar de utilizar una distribución normal utilice una de poisson.

**Ojo!, la distribución de Poisson acepta un único parámetro**

¿Has tenido que hacer transformaciones en los datos para llegar a alguna conclusión?


In [ ]:
import pymc3 as pm
import arviz as az
import numpy as np

with pm.Model() as model:
    dist_1 = pm.Uniform('dist_1', lower=5, upper=15)
    dist_2 = pm.Poisson('dist_2',10)
    
    delta = pm.Deterministic('delta', dist_2 - dist_1)
    
    trace = pm.sample(return_inferencedata=True)

    az.plot_posterior(trace, figsize=(16,8))
    
dibujar_muestras(trace, ['dist_1', 'dist_2', 'delta'])

# Detectar parámetros de una distribución

Para estos ejercicios partiremos de distribuciones que iremos generando a mano y trataremos de comprobar si el ajuste que hace PyMC3 es bueno.

## Ejemplo 3 - Obtener parámetros de una distribución uniforme

Para determinar los parámetros de una distribución necesitamos dos cosas, por una parte la distribución que creemos que siguen los datos y, por otra, un generador de números aleatorios que vaya probando valores.

Nosotros partimos con ventaja, sabemos que vamos a generar una distribución de datos con un generador uniforme entre 0 y 10.

Por lo tanto, le vamos a pedir a PyMC3 que detecte los parámetros para un generador uniforme.

Tenemos que decirle a PyMC3 cómo puede generar valores que probar en la distribución.

Esto son los parámetros **p_A** y **p_B**. Fijáos como los utilizamos más abajo.
1. p_A se va a mover entre -5 y 5
2. p_B se va a mover entre 5 y 15

Por qué esos valores? podríamos obtenerlos de un histograma, nos interesa dejar cierto margen de beneficio y no ir directos al mínimo y al máximo de la distribución para que "la magia bayesiana" pueda trabajar mejor con la incertidumbre, de esta forma sabremos no sólo si los parámetros originales están entre 0 y 10 sino también, la probabilidad de que hayan sido, por ejemplo, -1 y 11.

Fijaos también en el uso de **pm.sample_posterior_predictive** esa parte lo que hace es extraer muestras del resultado de nuestra inferencia.

In [ ]:
import pymc3 as pm
import arviz as az
import numpy as np

# Obtenemos 1000 medidas de una distribución uniforme entre 0 y 10
datos = np.random.uniform(10, 20, 1000)

with pm.Model() as model:
    
    p_A = pm.Uniform('p_A', lower=5, upper=15)
    p_B = pm.Uniform('p_B', lower=15, upper=25)
    
    obs = pm.Uniform('obs', lower=p_A, upper=p_B, observed=datos)
    
    trace = pm.sample(return_inferencedata=True)

    res = pm.sample_posterior_predictive(trace, samples=1000)
        
    az.plot_posterior(trace, figsize=(16,8))
    
dibujar_muestras(trace, ['p_A', 'p_B'], res, datos)

## Ejemplo 4 - Qué pasa si ajustamos a una normal?

In [ ]:
import pymc3 as pm
import arviz as az
import numpy as np

# Obtenemos 1000 medidas de una distribución uniforme entre 0 y 10
datos = np.random.uniform(10, 20, 1000)


with pm.Model() as model:
    # Como se trata de ajustar a una normal vamos a probar con media entre 10 y 20  y desviación entre 0 y 5
    p_A = pm.Uniform('p_A', lower=10, upper=20)
    p_B = pm.Uniform('p_B', lower=0, upper=5)
    
    obs = pm.Normal('obs', p_A, p_B, observed=datos)
    
    trace = pm.sample(return_inferencedata=True)

    res = pm.sample_posterior_predictive(trace, samples=1000)
        
    az.plot_posterior(trace, figsize=(16,8))
    
dibujar_muestras(trace, ['p_A', 'p_B'], res, datos)

En este caso vemos que las distribuciones no se parecen en nada si comparamos la original (en naranja) con la resultante (en verde), más allá de la altura de las dos distribuciones (que lo determina el número de observaciones), lo que está mal es la forma en sí misma.


## Ejercicio 4 - Ajuste a una normal

Ahora vamos a realizar el ajuste de una distribución normal a una normal


In [ ]:
import pymc3 as pm
import arviz as az
import numpy as np

# Obtenemos 1000 medidas de una distribución uniforme entre 0 y 10
datos = np.random.normal(20, 3, 1000)


with pm.Model() as model:
    # Como se trata de ajustar a una normal vamos a probar con media entre 10 y 20  y desviación entre 0 y 5
    p_A = pm.Uniform('p_A', lower=15, upper=25)
    p_B = pm.Uniform('p_B', lower=0, upper=5)
    
    obs = pm.Normal('obs', p_A, p_B, observed=datos)
    
    trace = pm.sample(return_inferencedata=True)

    res = pm.sample_posterior_predictive(trace, samples=1000)
        
    az.plot_posterior(trace, figsize=(16,8))
    
dibujar_muestras(trace, ['p_A', 'p_B'], res, datos)

In [ ]:
dibujar_muestras(trace, ['p_x', 'p_y'], res, datos)

## Ejercicio 5 - Ajuste a una distribución Gamma

Mismo ejercicio pero ajustando a una distribución Gamma.

Recordad que Gamma también tiene dos parámetros, k, que indica cómo de centrada está la distribución en el eje x y "zeta", que indica cómo de picuda es la distribución.

Probad con algunos valores para las distribuciones uniformes hasta que consideréis que se ha ajustado "bien" la distribución

In [ ]:
import pymc3 as pm
import arviz as az
import numpy as np

# Obtenemos 1000 medidas de una distribución uniforme entre 0 y 10
datos = np.random.normal(20, 3, 5000)


with pm.Model() as model:
    # Como se trata de ajustar a una normal vamos a probar con media entre 10 y 20  y desviación entre 0 y 5
    p_A = pm.Uniform('p_A', lower=, upper=)
    p_B = pm.Uniform('p_B', lower=, upper=)
    
    obs = pm.Gamma('obs', p_A, p_B, observed=datos)
    
    trace = pm.sample(return_inferencedata=True)

    res = pm.sample_posterior_predictive(trace, samples=1000)
        
    az.plot_posterior(trace, figsize=(16,8))
    
dibujar_muestras(trace, ['p_A', 'p_B'], res, datos)